#**BUKIT VISTA - GUEST REQUEST MANAGEMENT - COMPANY CAPSTONE PROJECT**

Name : Mochammad Dzahwan Fadhloly

ID Bangkit : M200D4KY1964

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE
import tensorflow as tf
import re
import string
from tensorflow.keras.layers import TextVectorization
import json

In [2]:
!pip install tensorflow tensorflowjs keras


In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path4 = ('/content/drive/MyDrive/Bangkit Bukit Vista/hermes_process_logs.csv')

In [6]:
hermes_process_logs = pd.read_csv(path4)
hermes_process_logs.head(100)

,id,user_id,agent_name,last_message,final_answer,escalation_status,prompt_arguments,prompt_response,state_id,state_name,...,category_name,category_description,category_summary,category_actions,tool_id,tool_name,tool_arguments,tool_response,created_at,updated_at
0,13341,6282297999004@c.us,Hermes,Noted Mbak,"Good morning, I've received the photos and the...",1.0,"{""user"": ""partner-6282297999004@c.us-20240423-...","{""data"": {""cost"": 56.625, ""completion"": {""inde...",1.0,goal-setting,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-23 00:03:20,2024-04-23 00:04:56
1,13341,6282297999004@c.us,Hermes,Noted Mbak,"Good morning, I've received the photos and the...",1.0,"{""user"": ""partner-6282297999004@c.us-20240423""...","{""data"": {""cost"": 995.1, ""completion"": {""index...",3.0,generate-answer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-23 00:03:20,2024-04-23 00:04:56
2,13341,6282297999004@c.us,Hermes,Noted Mbak,"Good morning, I've received the photos and the...",1.0,"{""user"": ""partner-6282297999004@c.us-20240423""...","{""data"": {""cost"": 60.465, ""completion"": {""inde...",4.0,validate-answer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-04-23 00:03:20,2024-04-23 00:04:56
3,13342,HMMWZJPTAK,Hosting,"Hi, \nCan you please respond to my request abo...","Hi Sergey,\n\nThank you for reaching out with ...",2.0,"{""user"": ""guest-confirmed-HMMWZJPTAK-202404230...","{""data"": {""cost"": 39.1575, ""completion"": {""ind...",1.0,goal-setting,...,request_extension,For request messages that intent to extend the...,Sergey has requested a response regarding exte...,Handle the request to extend the booking durat...,4.0,check_property_calendar,"{""room_id"": 397675, ""check_in_date"": ""2024-04-...","{'information': ""Currently, the property that ...",2024-04-23 00:03:46,2024-04-23 00:04:24
4,13342,HMMWZJPTAK,Hosting,"Hi, \nCan you please respond to my request abo...","Hi Sergey,\n\nThank you for reaching out with ...",2.0,"{""user"": ""guest-confirmed-HMMWZJPTAK"", ""model""...","{""data"": {""cost"": 33.6825, ""completion"": {""ind...",2.0,tool-calling,...,request_extension,For request messages that intent to extend the...,Sergey has requested a response regarding exte...,Handle the request to extend the booking durat...,4.0,check_property_calendar,"{""room_id"": 397675, ""check_in_date"": ""2024-04-...","{'information': ""Currently, the property that ...",2024-04-23 00:03:46,2024-04-23 00:04:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,13356,HMXBQ94TWH,Hosting,Okay thank you,"You're welcome, Natalie! I'm glad I could assi...",1.0,"{""user"": ""guest-confirmed-HMXBQ94TWH"", ""model""...","{""data"": {""cost"": 955.05, ""completion"": {""inde...",3.0,generate-answer,...,query_property_information,For messages that asking property information ...,Natalie is inquiring about the availability of...,"Confirm availability of bathroom amenities, sp...",6.0,get_property_capacity_information,"{""room_id"": ""397675"", ""property_name"": ""Dream ...",{'information': 'You will have a private room ...,2024-04-23 00:41:57,2024-04-23 00:42:23
96,13356,HMXBQ94TWH,Hosting,Okay thank you,"You're welcome, Natalie! I'm glad I could assi...",1.0,"{""user"": ""guest-confirmed-HMXBQ94TWH"", ""model""...","{""data"": {""cost"": 58.357499999999995, ""complet...",4.0,validate-answer,...,query_property_information,For messages that asking property information ...,Natalie is inquiring about the availability of...,"Confirm availability of bathroom amenities, sp...",1.0,get_property_information,"{""query"": ""bathroom amenities"", ""room_id"": ""39...","{'information': ""The bathroom is ensuite with ...",2024-04-23 00:41:57,2024-04-23 00:42:23
97,13356,HMXBQ94TWH,Hosting,Okay thank you,"You're welcome, Natalie! I'm glad I could assi...",1.0,"{""user"": ""guest-confirmed-HMXBQ94TWH"", ""model""...","{""data"": {""cost"": 58.357499999999995, ""complet...",4.0,validate-answer,...,query_property_information,For messages t

In [7]:
data_cleaned = hermes_process_logs.dropna()

In [8]:
df_new = data_cleaned[['last_message', 'category_name']]

# Simpan dataset baru ke file CSV
df_new.to_csv('new_hermes.csv', index=False)

# Verifikasi penyimpanan
print("Dataset baru telah disimpan ke file 'new_dataset.csv'")

Dataset baru telah disimpan ke file 'new_dataset.csv'


In [9]:
# Text preprocessing function
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text, re.I|re.A)
    text = text.lower()
    text = text.strip()
    return text


In [10]:
data = pd.read_csv('new_hermes.csv')
data.head(100)

,last_message,category_name
0,"Hi, \nCan you please respond to my request abo...",request_extension
1,"Hi, \nCan you please respond to my request abo...",request_extension
2,"Hi, \nCan you please respond to my request abo...",request_extension
3,"Hi, \nCan you please respond to my request abo...",request_extension
4,baik ..\nterimakasih kak,request_guest_size_alteration
...,...,...
95,"Merci encore!\nOui encore une chose, pour le p...",request_extra_service
96,"Merci encore!\nOui encore une chose, pour le p...",request_extra_service
97,"Merci encore!\nOui encore une chose, pour le p...",request_extra_service
98,"Merci encore!\nOui encore une chose, pour le p...",request_extra_service


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   last_message   79023 non-null  object
 1   category_name  79023 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [12]:
data['cleaned_message'] = data['last_message'].apply(clean_text)
data['onsite'] = data['category_name'] == 'request_onsite'

In [13]:
data['onsite'].value_counts()

onsite
False    70035
True      8988
Name: count, dtype: int64

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   last_message     79023 non-null  object
 1   category_name    79023 non-null  object
 2   cleaned_message  79023 non-null  object
 3   onsite           79023 non-null  bool  
dtypes: bool(1), object(3)
memory usage: 1.9+ MB


In [15]:
# Tokenize text
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(data['cleaned_message'])

X = tokenizer.texts_to_sequences(data['cleaned_message'])
X = pad_sequences(X, maxlen=200, padding='post')

y = data['onsite'].astype(int)


In [16]:
import pickle

with open('tokenizer_config.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)
    print("tokenizer saved")



tokenizer saved


In [17]:
# Pisahkan data menjadi train dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
# Terapkan SMOTE hanya pada set pelatihan
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [19]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_res, y_train_res)).shuffle(buffer_size=1024).batch(32)
val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)


In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=128, input_length=200),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Latih model
history = model.fit(X_train_res, y_train_res, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
2804/2804 ━━━━━━━━━━━━━━━━━━━━ 127s 41ms/step - accuracy: 0.8204 - loss: 0.3739 - val_accuracy: 0.8326 - val_loss: 0.2906
Epoch 2/10
2804/2804 ━━━━━━━━━━━━━━━━━━━━ 149s 46ms/step - accuracy: 0.9192 - loss: 0.1704 - val_accuracy: 0.8856 - val_loss: 0.2673
Epoch 3/10
2804/2804 ━━━━━━━━━━━━━━━━━━━━ 137s 44ms/step - accuracy: 0.9294 - loss: 0.1425 - val_accuracy: 0.8837 - val_loss: 0.2566
Epoch 4/10
2804/2804 ━━━━━━━━━━━━━━━━━━━━ 126s 39ms/step - accuracy: 0.9351 - loss: 0.1308 - val_accuracy: 0.9415 - val_loss: 0.2398
Epoch 5/10
2804/2804 ━━━━━━━━━━━━━━━━━━━━ 153s 42ms/step - accuracy: 0.9376 - loss: 0.1232 - val_accuracy: 0.9395 - val_loss: 0.2340
Epoch 6/10
2804/2804 ━━━━━━━━━━━━━━━━━━━━ 133s 39ms/step - accuracy: 0.9389 - loss: 0.1186 - val_accuracy: 0.9031 - val_loss: 0.2616
Epoch 7/10
2804/2804 ━━━━━━━━━━━━━━━━━━━━ 139s 38ms/step - accuracy: 0.9384 - loss: 0.1166 - val_accuracy: 0.9420 - val_loss: 0.2441
Epoch 8/10
2804/2804 ━━━━━━━━━━━━━━━━━━━━ 143s 38ms/step - accuracy: 

In [22]:
# Evaluasi model pada data uji
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {accuracy}")

494/494 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.9451 - loss: 0.1065
Test Accuracy: 0.9440683126449585


In [23]:

print(tf.__version__)

2.16.1


In [24]:
saved_model_path = "./my_model.h5"

model.save(saved_model_path)

In [25]:
!tensorflowjs_converter --input_format keras ./my_model.h5 ./model_json

2024-06-13 18:12:51.619714: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
failed to lookup keras version from the file,
    this is likely a weight only file


In [26]:
import json
# Simpan tokenizer ke dalam file JSON
tokenizer_json = tokenizer.to_json()
with open("tokenizer.json","w") as json_file:
  json_file.write(tokenizer_json)
print("Tokenizer has been saved to tokenizer.json")

Tokenizer has been saved to tokenizer.json


In [30]:
def inference(message, tokenizer, model, maxlen=200):
    """
    Perform inference to predict the category of the message.

    Parameters:
    - message: str, the input message
    - tokenizer: the tokenizer used to preprocess the text
    - model: the trained model used for prediction
    - maxlen: int, the maximum length of the sequence for padding (default: 200)

    Returns:
    - str, the predicted category ('Onsite' or 'Not Onsite')
    """
    # Preprocess the new message
    message = clean_text(message)

    # Tokenize the new message
    message_seq = tokenizer.texts_to_sequences([message])
    message_pad = pad_sequences(message_seq, maxlen=maxlen, padding='post')

    # Predict the category
    prediction = model.predict(message_pad)
    prediction = 'Onsite' if prediction > 0.5 else 'Not Onsite'

    return prediction

In [31]:
messages = [
    # Category: request_onsite
    "The room safe is not opening, could you send someone to assist?",
    "The Wi-Fi in our room is not working, can someone fix it?",
    "There's no hot water in the shower, can you send maintenance?",
    "The lights in the hallway are flickering, could you send someone to check it?",

    # Category: request_extra_service
    "Can we get extra towels in our room?",
    "I'd like to request a baby crib for our room.",
    "Can you arrange for a romantic dinner setup on the beach?",

    # Category: request_early_check_in
    "We have an early flight, can we check in as soon as we arrive?",
    "Is it possible to check in early if our room is ready?",
    "We arrive early in the morning, can we check in at 8 AM?",

    # Category: query_property_information
    "What dining options are available at the hotel?",
    "Can you tell me more about the facilities at the hotel?",
    "Can you tell me about the gym facilities?",

    # Category: information_guest_id
    "My reservation number is 98765, please confirm it.",
    "Guest ID: 54321, can you verify my booking details?"
]

# Perform inference for each message
results = []
for message in messages:
    prediction = inference(message, tokenizer, model)
    results.append((message, prediction))

# Display the results
for message, prediction in results:
    print(f"Message: {message}")
    print(f"Predicted Category: {prediction}")
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Message: The room safe is not opening, could you send someone to assist?
Predicted Category: Onsite

Message: The Wi-Fi in our room is not working, can someone fix it?
Predicted Category: Onsite

Message: There's no hot water in the shower, can you send maintenance?
Predicted Category: Onsite

Message: The lights in the hallway are flickering, could you send someone to check it?
Predicted Category: Onsite

Message: Can we get